In [1]:
import findspark
import os
findspark.init('/opt/spark')
java8_location= '/usr/lib/jvm/java-1.8.0-openjdk-1.8.0.252.b09-2.el8_1.x86_64'

os.environ['JAVA_HOME'] = java8_location

In [2]:
from pyspark import SparkContext, SparkConf
import pyspark
conf = SparkConf().setAppName("Ruihong").set("spark.executor.memory", "2g").set("spark.driver.memory", "200g")
conf = conf.set("spark.worker.timeout", "250000").set("spark.maxRemoteBlockSizeFetchToMem", "5M").set("spark.network.timeout", "300000").set("spark.executor.heartbeatInterval", "200000")
#.set("spark.maxRemoteBlockSizeFetchToMem", "5M")
sc = SparkContext.getOrCreate(conf)
#sc.setLogLevel("ERROR")
#sc =SparkContext(appName="Ruihong")

In [3]:
print(sc._conf.getAll())
import geopandas as gpd
import pyspark.sql.functions as F
from pyspark.sql.functions import *
from pyspark.sql import *
from pyspark.sql.types import *
import urllib
import os
from matplotlib import pyplot as plt
import geopy.distance
import pandas as pd
import numpy as np
import geopandas as gpd
from shapely.geometry import *
import shapely.speedups
import time
shapely.speedups.enable() # this makes some spatial queries run faster


[('spark.executor.memory', '2g'), ('spark.executor.heartbeatInterval', '200000'), ('spark.executor.id', 'driver'), ('spark.app.name', 'Ruihong'), ('spark.app.id', 'local-1588954733591'), ('spark.driver.host', 'datahub.geos.tamu.edu'), ('spark.maxRemoteBlockSizeFetchToMem', '5M'), ('spark.rdd.compress', 'True'), ('spark.serializer.objectStreamReset', '100'), ('spark.driver.port', '35477'), ('spark.driver.memory', '200g'), ('spark.master', 'local[*]'), ('spark.submit.deployMode', 'client'), ('spark.worker.timeout', '250000'), ('spark.network.timeout', '300000'), ('spark.ui.showConsoleProgress', 'true')]


In [4]:
spark = SparkSession.builder.getOrCreate()
Trip_to_merge1 = spark.read.csv("/vol_fast/tti/ruihongPro/TripRecordsReportTrips_Oct1.csv", header = False)
Trip_to_merge2 = spark.read.csv("/vol_fast/tti/ruihongPro/TripRecordsReportTrips_Oct2.csv", header = False)
Trip_to_merge3 = spark.read.csv("/vol_fast/tti/ruihongPro/TripRecordsReportTrips_Feb1.csv", header = False)
Trip_to_merge4 = spark.read.csv("/vol_fast/tti/ruihongPro/TripRecordsReportTrips_Feb2.csv", header = False)
Trip_to_merge5 = spark.read.csv("/vol_fast/tti/ruihongPro/TripRecordsReportTrips_June.csv", header = False)
Trip_to_merge6 = spark.read.csv("/vol_fast/tti/ruihongPro/TripRecordsReportTrips_July.csv", header = False)
Trip_to_merge = Trip_to_merge1.unionAll(Trip_to_merge2)
Trip_to_merge = Trip_to_merge.unionAll(Trip_to_merge3)
Trip_to_merge = Trip_to_merge.unionAll(Trip_to_merge4)
Trip_to_merge = Trip_to_merge.unionAll(Trip_to_merge5)
Trip_to_merge = Trip_to_merge.unionAll(Trip_to_merge6)
Trip_to_merge = Trip_to_merge.withColumnRenamed("_c0", "TripId").withColumnRenamed("_c1", "DeviceId").withColumnRenamed("_c2", "ProviderId").withColumnRenamed("_c3", "Mode").withColumnRenamed("_c4", "StartDate")
Trip_to_merge = Trip_to_merge.withColumnRenamed("_c5", "StartWDay").withColumnRenamed("_c6", "EndDate").withColumnRenamed("_c7", "EndWDay").withColumnRenamed("_c8", "StartLocLat").withColumnRenamed("_c9", "StartLocLon")
Trip_to_merge = Trip_to_merge.withColumnRenamed("_c10", "EndLocLat").withColumnRenamed("_c11", "EndLocLon").withColumnRenamed("_c12", "IsStartHome").withColumnRenamed("_c13", "IsEndHome").withColumnRenamed("_c14", "GeospatialType")
Trip_to_merge = Trip_to_merge.withColumnRenamed("_c15", "ProviderType").withColumnRenamed("_c16", "ProviderDrivingProfile").withColumnRenamed("_c17", "VehicleWeightClass").withColumnRenamed("_c18", "FirstZoneName").withColumnRenamed("_c19", "LastZoneName")
Trip_to_merge = Trip_to_merge.withColumnRenamed("_c20", "MultipleZones").withColumnRenamed("_c21", "MultipleCorridors")
Trip_to_merge = Trip_to_merge.select(['DeviceId', 'ProviderId'])

In [7]:
ProviderId = Trip_to_merge.distinct().toPandas()

In [14]:
Trip_to_merge.distinct().count()

45

In [9]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import os
os.chdir("/vol_fast/tti/ruihongPro")

more = True
use_dev = False
use_cluster = False
use_sample = False
use_temporal= False
use_road = False
features = pd.read_csv('Device_all2.csv')


In [10]:
features.head()

,DeviceId,mean_detour_metrics,std_detour_metrics,mean_AvgTimeGra,std_AvgTimeGra,mean_AvgDistanceGra,std_AvgDistanceGra,mean_POInum,std_POInum,mean_TotalDistance,...,CountMax1,CountMax2,CountMax3,CountMax4,DistinctHour,TotalNumTrip,mean_average_speed,std_average_speed,mean_Total_time_interval,std_Total_time_interval
0,0003ded233bf8f81655e753b1d134261,0.648223,NaN,1.042166,NaN,0.008980,NaN,0.0,NaN,17.251403,...,1,NaN,NaN,NaN,1,1,0.014388,NaN,1199.000,NaN
1,0004128692d07990975d83294c8d9612,1.000000,NaN,93.738500,NaN,0.474652,NaN,0.0,NaN,0.949303,...,1,NaN,NaN,NaN,1,1,0.005064,NaN,187.477,NaN
2,00044e5e9542fb977837ac9984b7f73b,0.906792,NaN,179.570000,NaN,2.767001,NaN,0.0,NaN,55.340026,...,1,NaN,NaN,NaN,1,1,0.015409,NaN,3591.400,NaN
3,0004ba1b077d074af9c5b9993105882a,0.856173,0.157257,1.020458,0.016688,0.005193,0.000303,0.0,0.0,0.826438,...,1,1.0,NaN,NaN,2,2,0.005596,0.000453,151.500,94.045202
4,00052ffb5a222c6da495d4e31af669a8,0.936262,NaN,164.762556,NaN,0.849216,NaN,0.0,NaN,7.642944,...,1,NaN,NaN,NaN,1,1,0.005154,NaN,1482.863,NaN


In [12]:
# features = features.drop(['list'], axis = 1)
features.fillna({'HourMax1':'None', 'HourMax2':'None', 'HourMax3':'None', 'HourMax4':'None',
                 'CountMax1':0,  'CountMax2':0,  'CountMax3':0, 'CountMax4':0}, inplace=True)

features = pd.concat([features,pd.get_dummies(features['GeospatialType'])], axis = 1)

features['CountMax1'] = features['CountMax1']/features['TotalNumTrip']
features['CountMax2'] = features['CountMax2']/features['TotalNumTrip']
features['CountMax3'] = features['CountMax3']/features['TotalNumTrip']
features['CountMax4'] = features['CountMax4']/features['TotalNumTrip']


In [13]:
features.shape

(5451095, 38)

In [15]:
ProviderId.columns

Index(['DeviceId', 'ProviderId'], dtype='object')

In [16]:
features = pd.merge(features, ProviderId, on = 'DeviceId', how = 'inner')

In [17]:
features.columns

Index(['DeviceId', 'mean_detour_metrics', 'std_detour_metrics',
       'mean_AvgTimeGra', 'std_AvgTimeGra', 'mean_AvgDistanceGra',
       'std_AvgDistanceGra', 'mean_POInum', 'std_POInum', 'mean_TotalDistance',
       'std_TotalDistance', 'mean_point_num', 'std_point_num', 'min_profile',
       'max_profile', 'GeospatialType', 'HourMax1', 'HourMax2', 'HourMax3',
       'HourMax4', 'CountMax1', 'CountMax2', 'CountMax3', 'CountMax4',
       'DistinctHour', 'TotalNumTrip', 'mean_average_speed',
       'std_average_speed', 'mean_Total_time_interval',
       'std_Total_time_interval', 'EE', 'EI', 'IE', 'II', 'EE', 'EI', 'IE',
       'II', 'ProviderId'],
      dtype='object')

In [33]:
features[features[['mean_detour_metrics', 'mean_AvgTimeGra', 'mean_AvgDistanceGra', 'mean_TotalDistance', 'mean_point_num']].isnull().any(axis = 1)]

,DeviceId,mean_detour_metrics,std_detour_metrics,mean_AvgTimeGra,std_AvgTimeGra,mean_AvgDistanceGra,std_AvgDistanceGra,mean_POInum,std_POInum,mean_TotalDistance,...,std_Total_time_interval,EE,EI,IE,II,EE,EI,IE,II,ProviderId
147752,6fd1dc335d737df50f87d0bdd344d1c0,NaN,NaN,11.250000,NaN,0.0,NaN,1.0,NaN,0.0,...,NaN,0,0,0,1,0,0,0,1,47c1b025fa18ea96c33fbb6718688c0f
350588,e437ae1c3dcc835279f86d2dc5b4ac03,NaN,NaN,1.029412,NaN,0.0,NaN,1.0,NaN,0.0,...,NaN,0,0,0,1,0,0,0,1,3ad7c2ebb96fcba7cda0cf54a2e802f5
367525,835fbf68ac23d777ac4721c0d9d36d23,NaN,NaN,1.000000,NaN,0.0,NaN,0.0,NaN,0.0,...,NaN,0,0,0,1,0,0,0,1,e2c0be24560d78c5e599c2a9c9d0bbd2
724172,92dad3a5b9f861a81001062b1b193a7a,NaN,NaN,1.029412,NaN,0.0,NaN,1.0,NaN,0.0,...,NaN,0,0,0,1,0,0,0,1,e2c0be24560d78c5e599c2a9c9d0bbd2
854950,5af42faa9169370b3c3ebc18a80bfb64,NaN,NaN,0.833333,NaN,0.0,NaN,0.0,NaN,0.0,...,NaN,0,0,0,1,0,0,0,1,3ad7c2ebb96fcba7cda0cf54a2e802f5
951604,e109923be3096e42465d04b9372d8157,NaN,NaN,0.947368,NaN,0.0,NaN,0.0,NaN,0.0,...,NaN,0,0,0,1,0,0,0,1,e2c0be24560d78c5e599c2a9c9d0bbd2
952874,ed1c790d3dc796e5d4f8bb32d297d39a,NaN,NaN,0.977778,NaN,0.0,NaN,0.0,NaN,0.0,...,NaN,0,0,0,1,0,0,0,1,e2c0be24560d78c5e599c2a9c9d0bbd2
952943,edbdbf0d1abac95b919183c805f35c06,NaN,NaN,0.992958,NaN,0.0,NaN,0.0,NaN,0.0,...,NaN,0,0,0,1,0,0,0,1,3ad7c2ebb96fcba7cda0cf54a2e802f5
1436284,b089a7d0609405e9dde7b7f21580c584,NaN,NaN,1.000000,NaN,0.0,NaN,0.0,NaN,0.0,...,NaN,0,0,0,1,0,0,0,1,e2c0be24560d78c5e599c2a9c9d0bbd2
1467909,d8dcac9eab4d28b1f76cabf4f14fefe1,NaN,NaN,2.130435,NaN,0.0,NaN,1.0,NaN,0.0,...,NaN,0,0,0,1,0,0,0,1,e2c0be24560d78c5e599c2a9c9d0bbd2


In [36]:
from sklearn import preprocessing
normalized_features = preprocessing.normalize(features[['mean_detour_metrics', 'mean_AvgTimeGra', 'mean_AvgDistanceGra', 'mean_TotalDistance', 'mean_point_num']].dropna())


In [37]:
normalized_features = pd.DataFrame(normalized_features, columns = ['mean_detour_metrics', 'mean_AvgTimeGra', 'mean_AvgDistanceGra', 'mean_TotalDistance', 'mean_point_num'])

In [39]:
normalized_features[['DeviceId', 'ProviderId', 'min_profile']] = features[['DeviceId', 'ProviderId', 'min_profile']]

In [42]:
normalized_features.drop(['DeviceId'], axis = 1, inplace = True)

In [43]:
check = normalized_features.groupby('ProviderId').mean()

In [44]:
check1 = normalized_features.groupby('ProviderId').std()

In [20]:
check.columns

Index(['mean_detour_metrics', 'std_detour_metrics', 'mean_AvgTimeGra',
       'std_AvgTimeGra', 'mean_AvgDistanceGra', 'std_AvgDistanceGra',
       'mean_POInum', 'std_POInum', 'mean_TotalDistance', 'std_TotalDistance',
       'mean_point_num', 'std_point_num', 'min_profile', 'max_profile',
       'HourMax1', 'CountMax1', 'CountMax2', 'CountMax3', 'CountMax4',
       'DistinctHour', 'TotalNumTrip', 'mean_average_speed',
       'std_average_speed', 'mean_Total_time_interval',
       'std_Total_time_interval', 'EE', 'EI', 'IE', 'II', 'EE', 'EI', 'IE',
       'II'],
      dtype='object')

In [45]:
check[['mean_detour_metrics', 'mean_AvgTimeGra', 'mean_AvgDistanceGra', 'mean_TotalDistance', 'mean_point_num', 'min_profile']]

,mean_detour_metrics,mean_AvgTimeGra,mean_AvgDistanceGra,mean_TotalDistance,mean_point_num,min_profile
ProviderId,,,,,,
05049e90fa4f5039a8cadc6acbb4b2cc,0.005990,0.998689,0.003459,0.024214,0.044563,1
0bb4aec1710521c12ee76289d9440817,0.006604,0.725654,0.005069,0.136794,0.338616,1
16a5cdae362b8d27a1d8f8c7b78b4330,0.007503,0.763844,0.005449,0.134286,0.304586,1
26657d5ff9020d2abefe558796b99584,0.006913,0.751279,0.005358,0.139433,0.309918,3
310dcbbf4cce62f762a2aaa148d556bd,0.007318,0.765594,0.004957,0.124594,0.293825,1
320722549d1751cf3f247855f937b982,0.008386,0.680835,0.004991,0.167920,0.404148,1
371bce7dc83817b7893bcdeed13799b5,0.006585,0.758761,0.005377,0.146607,0.303138,3
3988c7f88ebcb58c6ce932b957b6f332,0.006974,0.759038,0.005339,0.140303,0.306034,3
3ad7c2ebb96fcba7cda0cf54a2e802f5,0.006923,0.740151,0.005206,0.131231,0.321537,1


In [46]:
check1[['mean_detour_metrics', 'mean_AvgTimeGra', 'mean_AvgDistanceGra', 'mean_TotalDistance', 'mean_point_num']]

,mean_detour_metrics,mean_AvgTimeGra,mean_AvgDistanceGra,mean_TotalDistance,mean_point_num
ProviderId,,,,,
05049e90fa4f5039a8cadc6acbb4b2cc,NaN,NaN,NaN,NaN,NaN
0bb4aec1710521c12ee76289d9440817,0.005742,0.391233,0.003686,0.189877,0.389144
16a5cdae362b8d27a1d8f8c7b78b4330,0.009805,0.368983,0.003845,0.184090,0.368130
26657d5ff9020d2abefe558796b99584,0.009067,0.377907,0.003900,0.191637,0.374975
310dcbbf4cce62f762a2aaa148d556bd,0.012287,0.378304,0.003915,0.185600,0.368644
320722549d1751cf3f247855f937b982,0.011655,0.391957,0.003648,0.208236,0.392843
371bce7dc83817b7893bcdeed13799b5,0.006877,0.372102,0.003801,0.195588,0.366657
3988c7f88ebcb58c6ce932b957b6f332,0.008439,0.372000,0.003806,0.190216,0.368664
3ad7c2ebb96fcba7cda0cf54a2e802f5,0.009052,0.388464,0.003865,0.185369,0.382272
